In [188]:
import pandas as pd
import os
import datetime as dt
import re
import xlsxwriter
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import seaborn as sns

In [189]:
def get_files(mmo_file_name, reg_file_name):
    oris_mmo = pd.read_csv(mmo_file_name, sep=';')
    oris_regions = pd.read_csv(reg_file_name, sep=';')
    oris = pd.concat([oris_mmo, oris_regions])
    return oris

In [190]:
def oris_filter(oris, week):
#     Добавляем новый столбец с номером ресторана
    oris["Ресторан"] = oris['Номер ресторана'].str[0:4]
#     Оставляем только нужные столбцы
    oris = oris[["Ресторан", 'Дата прохождения МО в клинике', 'ФИО пациента', 'Тип осмотра', 'Должность', "Телефон", 'Номер ресторана']]
#     Оставляем в таблице только новых сотрудников и нужные нам должности
    position = ['кассир', 'повар', 'член бригады ресторана ', 'тренер',
            'Помощник члена бригады ресторана', 'Уборщик (дневное время)', 'курьер']
    oris = oris[oris['Должность'].isin(position)]
    oris = oris[oris['Тип осмотра'] == 'Новый сотрудник']
# Фильтруем данные по дате    
    oris['Дата прохождения МО в клинике'] = pd.to_datetime(oris['Дата прохождения МО в клинике'], dayfirst=True)
    pass_mo = oris[oris['Дата прохождения МО в клинике'].isin(week)]
    not_pass_mo = oris[oris['Дата прохождения МО в клинике'].isnull()]
    sent_to_mo = pd.concat([pass_mo, not_pass_mo])
    return sent_to_mo, pass_mo

In [191]:
def save_excel_file(oris_filtred, dates):
        """
        Сохраняет таблицу в Excel файл
        """
        file_name = f"ОРИС общий {dates}.xlsx"
        with pd.ExcelWriter(file_name, 
                            engine='xlsxwriter',
                            datetime_format='dd.mm.yyyy') as writer:
            oris_filtred.to_excel(writer, sheet_name='Данные', index=False)
            print(f"Excel file {file_name} was created")

In [193]:
week = pd.date_range('2020.12.7', periods=7, freq="D")
data = get_files("ОРИС ММО 7.12-13.12.csv", "ОРИС Рег 7.12-13.12.csv")
all_sent_to_mo, have_passed_mo = oris_filter(data, week)
save_excel_file(all_sent_to_mo, "7.12-13.12")

In [197]:
have_passed_mo.groupby(['Ресторан']).aggregate('ФИО пациента').count()

Ресторан
0014    1
0025    1
0050    1
0055    1
0126    2
0127    1
0137    1
0226    1
0257    1
0341    1
0350    1
0394    1
0401    2
0403    3
0434    1
0440    1
0470    1
0511    1
0599    1
0624    1
0713    1
0893    1
0920    1
9044    2
Name: ФИО пациента, dtype: int64

In [198]:
all_sent_to_mo.groupby(['Ресторан']).aggregate('ФИО пациента').count()

Ресторан
0002    1
0014    1
0025    1
0035    1
0050    1
       ..
0893    1
0909    1
0920    1
0942    3
9044    2
Name: ФИО пациента, Length: 69, dtype: int64